# Run spec2 pipeline on rate files

Need to copy all the needed files over to the working directory.

**Author notes**:
- Anything I need to know on the spec2 steps?
- I have to try turning on the correction for contamination

## Imports

In [ ]:
# Packages that allow us to get information about objects:
import os
# Update to a path in your system (see details below at "Reference files")
os.environ["CRDS_PATH"] = "/Users/cpacifici/crds_cache_july2023/"
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"


In [ ]:
# The calwebb_spec and spec3 pipelines
from jwst.pipeline import Spec2Pipeline

# data models
from jwst import datamodels

# utility
from astropy.table import Table
import numpy as np
import json
import glob
import shutil

# jdaviz
from jdaviz import Mosviz

In [ ]:
import jwst
print('jwst',jwst.__version__)

In [ ]:
# Update appropriately
data_dir_in = "/Users/cpacifici/DATA/NGDEEP/rate/"
data_dir_images = "/Users/cpacifici/DATA/NGDEEP/image3/"
data_dir_asn_level2 = "/Users/cpacifici/DATA/NGDEEP/asn_level2/"
data_dir_out_spec2 = "/Users/cpacifici/DATA/NGDEEP/spec2/"

## Read list of rate files for imaging

In [ ]:
listrate_file = './list_ngdeep_rate.csv'
listrate = Table.read(listrate_file)
listrate[np.where((listrate['FILTER']=='GR150R') | (listrate['FILTER']=='GR150C'))]

## Run spec2 on all GR150R/C images using the provided association files

### Check an association file for level 2

In [ ]:
asnfile = data_dir_asn_level2 + 'jw02079-o004_20230622t175524_spec2_00001_asn.json'
asn_data = json.load(open(asnfile))
print(asn_data['products'][0]['members'][0]['expname'])
asn_data

### Copy all needed files to the current directory

In [ ]:
asn_spec2 = glob.glob(data_dir_asn_level2+'*spec2*')
for file in asn_spec2:
    shutil.copy(file, './')

In [ ]:
rate_clear = listrate[np.where((listrate['FILTER']=='GR150R') | (listrate['FILTER']=='GR150C'))]['FILENAME']
for file in rate_clear:
    shutil.copy(data_dir_in+file, './')

In [ ]:
catalogfiles = glob.glob(data_dir_images+'*cat.ecsv')
for file in catalogfiles:
    shutil.copy(file, './')

In [ ]:
imagefiles = glob.glob(data_dir_images+'*i2d.fits')
for file in imagefiles:
    shutil.copy(file, './')

In [ ]:
segfiles = glob.glob(data_dir_images+'*segm.fits')
for file in segfiles:
    shutil.copy(file, './')

## Run spec2

In [ ]:
asn_spec2_local = glob.glob('./jw02079-o004_*_spec2_*_asn.json')
asn_spec2_local

In [ ]:
for file in asn_spec2_local:
    print(file)
    spec2 = Spec2Pipeline() 
    result = spec2.call(file, 
                        steps={'bkg_subtract':{'skip':False},
                               'extract_2d':{'wfss_nbright':500,
                                             'wfss_extract_half_height': 25,
                                             },
                               'photom':{'skip':False},
                              },
                        save_results=True,
                        output_dir=data_dir_out_spec2)


### Delete files not needed anymore

In [ ]:
dir_to_be_del = './files_to_be_deleted'
if not os.path.exists(dir_to_be_del):
    os.mkdir(dir_to_be_del)
    
for file in asn_spec2_local:
    shutil.move(file, dir_to_be_del)
    
for file in glob.glob('./*rate.fits'):
    shutil.move(file, dir_to_be_del)

for file in glob.glob('./*niriss_clear*'):
    shutil.move(file, dir_to_be_del)

shutil.rmtree(dir_to_be_del)